In [9]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
# from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.support import expected_conditions as EC
from time import sleep
import gensim
import gensim.corpora as corpora
import nltk
import pyLDAvis
import pyLDAvis.gensim
import nltk
from nltk.corpus import wordnet, stopwords, words
from nltk.stem import WordNetLemmatizer

In [12]:
nltk.download('words')
stop_words = stopwords.words('english')

[nltk_data] Downloading package words to
[nltk_data]     /Users/thanawatthongpia/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [10]:
def preprocess_text(sentences):

    lemmatizer = WordNetLemmatizer()
    lemmatized_sentences = []
    bigram = gensim.models.Phrases(sentences, min_count=5, threshold=100)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    
    data_words_nostops = [[word for word in doc if word not in stop_words] for doc in sentences]
    data_words_bigrams = [bigram_mod[doc] for doc in data_words_nostops]

    for sentence in data_words_bigrams:
        lemmatized_tokens = [lemmatizer.lemmatize(token, wordnet.VERB) for token in sentence]
        lemmatized_sentences.append(lemmatized_tokens)
    return lemmatized_sentences

In [13]:
def import_text_to_list_of_lists(file_path, delimiter='\t'):
    result = []
    with open(file_path, 'r') as file:
        for line in file:
            line = line.strip()  # Remove leading/trailing whitespace and newlines
            sublist = line.split(delimiter)
            result.append(sublist)
    return result

In [11]:
def corpus_topic(corpus,num_topics=10):

    data_lemmatized = preprocess_text(corpus)
    id2word = corpora.Dictionary(data_lemmatized)  

    # Term Document Frequency 
    LDA_corpus = [id2word.doc2bow(text) for text in data_lemmatized]
    lda_model = gensim.models.ldamodel.LdaModel(corpus=LDA_corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)
    
    # # Compute Perplexity
    # print('\nPerplexity: ', lda_model.log_perplexity(LDA_corpus))  
    # # a measure of how good the model is. lower the better.

    # # Compute Coherence Score
    # coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
    # coherence_lda = coherence_model_lda.get_coherence()
    # print('Coherence Score: ', coherence_lda)
    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim.prepare(lda_model, LDA_corpus, id2word)
    return vis

In [3]:
#Scraping links on news page with click load more button

driver = webdriver.Safari()
driver.get('https://scbtechx.io/news/')
wait_duration = 5

while True:
    try:
        # Find the button element by its class
        load_more_button = driver.find_element(By.CLASS_NAME, "wpr-load-more-btn")
        driver.execute_script("arguments[0].scrollIntoView();", load_more_button)
        load_more_button.click()
        print("Clicked 'Load More' button")
        sleep(wait_duration)

    except ElementNotInteractableException:
        break

soup = BeautifulSoup(driver.page_source, 'html.parser')
section = soup.find("section", class_="wpr-grid elementor-clearfix grid-images-loaded")
a_tags = section.find_all("a")

keep_links = [link.get('href') for link in a_tags]
news_lisks = [link for link in set(keep_links) if link.startswith('https://scbtechx.io/') and 'https://scbtechx.io/category/' not in link]
driver.quit()

Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button
Clicked 'Load More' button


In [18]:
news_lisks[:10]

['https://scbtechx.io/news/pointxnews/news-scb-techx-pointx-launch/#respond',
 'https://scbtechx.io/news/pointxnews/enjoy-the-freedom-of-point-for-cash-redemption-with-the-pointx-new-year-giveaway/#respond',
 'https://scbtechx.io/news/others/news_scb_techx_internship_program_2023/',
 'https://scbtechx.io/news/others/news_scb_techx_internship_program_2023/#respond',
 'https://scbtechx.io/news/pointxnews/celebrate-the-festive-seasons-with-scan-pay-from-pointx/#respond',
 'https://scbtechx.io/news/leadershipnews/scb-techx-jonathan-sharp-cto/',
 'https://scbtechx.io/news/pointxnews/individuals-can-now-pay-their-income-tax-with-pointx-instead-of-cash/#respond',
 'https://scbtechx.io/news/others/track-interview-for-business-analyst-system-analyst/',
 'https://scbtechx.io/news/pointxnews/pointx-launches-extravagant-add-to-cart-at-x-store-campaign/#respond',
 'https://scbtechx.io/news/pointxnews/dear-coffee-lovers-dont-miss-pointxs-let-coffee-lovers-enjoy-coffee-every-day/']

In [17]:
techx_corpus = import_text_to_list_of_lists('src/Corpus_TechX.txt', delimiter='\t')
corpus_topic(preprocess_text(techx_corpus))


Perplexity:  -6.684810601212469
Coherence Score:  0.43309610510121493


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5     -0.290689  0.108397       1        1  16.719114
7     -0.305277  0.027804       2        1  13.192491
2     -0.045961 -0.398084       3        1  12.570399
1      0.058823  0.019101       4        1  12.434667
3      0.043349  0.151301       5        1  10.949949
8      0.149864  0.014750       6        1  10.374636
9      0.133667  0.043595       7        1   8.851225
6      0.133800  0.048923       8        1   6.328677
0      0.047617 -0.022675       9        1   4.967797
4      0.074806  0.006887      10        1   3.611044, topic_info=          Term        Freq       Total Category  logprob  loglift
1132     point  777.000000  777.000000  Default  30.0000  30.0000
18         scb  609.000000  609.000000  Default  29.0000  29.0000
869     pointx  811.000000  811.000000  Default  28.0000  28.0000
834      techx  363.000000  363.000000  Default  27.0000  27.0000
448    digital  295.000000  295.000000  Default  26.0000  26.0000
...        ...         ...         ...      ...      ...      ...
1782     roles    5.750574    6.703337  Topic10  -5.1734   3.1679
1907     phase    5.732428    6.684995  Topic10  -5.1766   3.1674
75     process   15.838060   22.745829  Topic10  -4.1603   2.9592
115     credit   38.522693   89.019052  Topic10  -3.2715   2.4836
858   solution   10.681509   34.842405  Topic10  -4.5542   2.1389

[399 rows x 6 columns], token_table=      Topic      Freq          Term
term                               
149       8  0.946366          able
200       6  0.927438       account
1747      1  0.981170    accumulate
1124      2  0.987829  accumulation
1152      4  0.559325        across
...     ...       ...           ...
616       8  0.330556          work
616       9  0.157408          work
1151      2  0.991535         world
1681      4  0.952868          year
1304      3  0.959494         years

[415 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 8, 3, 2, 4, 9, 10, 7, 1, 5])